In [1]:
import os, time, shutil
from pathlib import Path
from glob import glob
import numpy as np
import spikeinterface.full as si

In [2]:
def clean_sorting(rec, save_root, stream_name):
    
    # Creates output folder, sorts and clears large temporary files
    save_path = os.path.join(save_root,stream_name)
    output_folder = Path(os.path.join(save_path, 'sorted'))
    
    if not os.path.exists(os.path.join(output_folder,'amplitudes.npy')):
        output_folder.mkdir(parents=True, exist_ok=True)
        raw_file = os.path.join(output_folder,'recording.dat')
        wh_file = os.path.join(output_folder, 'temp_wh.dat')
        
        print(f"DURATION: {rec.get_num_frames() / rec.get_sampling_frequency()} s -- "
                f"NUM. CHANNELS: {rec.get_num_channels()}")
        
        try:
            t_start_sort = time.time()
            sorting = si.run_sorter(sorter, rec, output_folder=output_folder, verbose=True,
                                    **sorter_params)
            print(f"\n\nSpike sorting elapsed time {time.time() - t_start_sort} s")
            
            #Making sure we clean up the temporary .npy files
            if os.path.exists(wh_file):
                os.remove(wh_file)
            if os.path.exists(raw_file):
                os.remove(raw_file)
        except Exception:
            if os.path.exists(wh_file):
                os.remove(wh_file)
            if os.path.exists(raw_file):
                os.remove(raw_file)

In [3]:
sorter = 'kilosort2_5'
si.Kilosort2_5Sorter.set_kilosort2_5_path('/home/phornauer/Git/Kilosort_2020b')
sorter_params = si.get_default_sorter_params(si.Kilosort2_5Sorter)
sorter_params['n_jobs'] = -1
sorter_params['detect_threshold'] = 5.5
sorter_params['minFR'] = 0.01
sorter_params['minfr_goodchannels'] = 0.01
sorter_params['keep_good_only'] = False
sorter_params['do_correction'] = False

Setting KILOSORT2_5_PATH environment variable for subprocess calls to: /home/phornauer/Git/Kilosort_2020b


In [10]:
N_recordings = 5 # Number of recordings per chip/well that need to exist
parent_path = '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/'
recording_date = '230*'
project_name = 'How_medium_change'
chip_id = 'M05569'

assay_name = 'Network'
assay_id = '00*'
file_name = 'data.raw.h5'

path_parts = [parent_path, recording_date, project_name, chip_id]
path_pattern = os.path.join(*path_parts)
path_list  = glob(path_pattern)
chip_ids = [path.split('/')[-1] for path in path_list]

['/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230125/How_medium_change/M05569',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230126/How_medium_change/M05569',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230123/How_medium_change/M05569',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230124/How_medium_change/M05569']

In [12]:
chip_file_list

['/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230123/How_medium_change/M05569/Network/000006/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230123/How_medium_change/M05569/Network/000017/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230124/How_medium_change/M05569/Network/000023/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230125/How_medium_change/M05569/Network/000029/data.raw.h5',
 '/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/230126/How_medium_change/M05569/Network/000035/data.raw.h5']

In [17]:
for chip in set(chip_ids):
    chip_parts = [parent_path, recording_date, project_name, chip, assay_name, assay_id, file_name]
    chip_pattern = os.path.join(*chip_parts)
    chip_file_list  = glob(chip_pattern)
    chip_file_list.sort()
    
    for well_id in range(3): # Iterate over all wells
        stream_name = 'well00' + str(well_id)
        
        path_parts = chip_file_list[0].split('/')
        path_parts[0] = '/'
        path_parts[6] = 'intermediate_data'
        save_root = os.path.join(*path_parts[0:12])
        
        well_recording_list = []
        
        for rec_path in chip_file_list: #Iterate over recordings to be concatenated 
            
            try: # If not all wells were recorded, should be the only cause for an error
                rec = si.MaxwellRecordingExtractor(rec_path,stream_name=stream_name)
                well_recording_list.append(rec)
            except Exception:
                continue
        
        if len(well_recording_list) >= N_recordings:
            multirecording = si.concatenate_recordings(well_recording_list[:N_recordings])
            clean_sorting(multirecording, save_root, stream_name)

DURATION: 4500.22 s -- NUM. CHANNELS: 1020
RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptvyq5d8m8/script.sh


write_binary_recording:   0%|          | 0/4501 [00:00<?, ?it/s]

RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptz4f_2flg/script.sh
RUNNING SHELL SCRIPT: /tmp/tmp_shellscript53l67uvv/script.sh
RUNNING SHELL SCRIPT: /net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/230123/How_medium_change/M05569/well000/sorted/run_kilosort2_5.sh


                                                                              < M A T L A B (R) >

                                                                    Copyright 1984-2020 The MathWorks, Inc.

                                                                    R2020b (9.9.0.1467703) 64-bit (glnxa64)

                                                                                August 26, 2020



 

To get started, type doc.

For product information, visit www.mathworks.com.

 

Time   0s. Computing whitening matrix.. 

Getting channel whitening matrix... 

Channel-whitening matrix computed. 

Time  16s. Loading raw data and applying filters... 

Time 453s. Finished preprocessing 687 bat

write_binary_recording:   0%|          | 0/4501 [00:00<?, ?it/s]

RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptfah1eky2/script.sh
RUNNING SHELL SCRIPT: /tmp/tmp_shellscript8bos9upb/script.sh
RUNNING SHELL SCRIPT: /net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/230123/How_medium_change/M05569/well001/sorted/run_kilosort2_5.sh


                                                                              < M A T L A B (R) >

                                                                    Copyright 1984-2020 The MathWorks, Inc.

                                                                    R2020b (9.9.0.1467703) 64-bit (glnxa64)

                                                                                August 26, 2020



 

To get started, type doc.

For product information, visit www.mathworks.com.

 

Time   0s. Computing whitening matrix.. 

Getting channel whitening matrix... 

Channel-whitening matrix computed. 

Time  13s. Loading raw data and applying filters... 

Time 464s. Finished preprocessing 687 bat

write_binary_recording:   0%|          | 0/4501 [00:00<?, ?it/s]

RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptn0ap9gqx/script.sh
RUNNING SHELL SCRIPT: /tmp/tmp_shellscriptzqk0m1lm/script.sh
RUNNING SHELL SCRIPT: /net/bs-filesvr02/export/group/hierlemann/intermediate_data/Mea1k/phornauer/230123/How_medium_change/M05569/well002/sorted/run_kilosort2_5.sh


                                                                              < M A T L A B (R) >

                                                                    Copyright 1984-2020 The MathWorks, Inc.

                                                                    R2020b (9.9.0.1467703) 64-bit (glnxa64)

                                                                                August 26, 2020



 

To get started, type doc.

For product information, visit www.mathworks.com.

 

Time   0s. Computing whitening matrix.. 

Getting channel whitening matrix... 

Channel-whitening matrix computed. 

Time  13s. Loading raw data and applying filters... 

Time 453s. Finished preprocessing 687 bat

In [22]:
well_id = 0
stream_name = 'well00' + str(well_id)
        
path_parts = chip_file_list[0].split('/')
path_parts[0] = '/'
path_parts[6] = 'intermediate_data'
save_root = os.path.join(*path_parts[0:12])
well_recording_list = []
for rec_path in chip_file_list: #Iterate over recordings to be concatenated 
            
    try: # If not all wells were recorded, should be the only cause for an error
        rec = si.MaxwellRecordingExtractor(rec_path,stream_name=stream_name)
        well_recording_list.append(rec)
    except Exception:
        continue

In [16]:
list(range(3))


[0, 1, 2]